In [1]:
# 載入套件
import requests
from bs4 import BeautifulSoup
# 為了使用quote function改變關鍵字的格式
import urllib.parse
import csv
import sqlite3
import re

In [2]:
def main():
    query = str(input('輸入商品：'))
    # 關鍵字轉成url的格式(ASCII)
    q = urllib.parse.quote(query)
    print('query url: {}'.format(q))
    # e.g. https://ezprice.com.tw/s/nitendo%20switch/price/

    page = requests.get('https://ezprice.com.tw/s/' + q + '/price/', verify=False).text
    soup = BeautifulSoup(page, 'lxml')
    items = list()

    for div in soup.find_all('div', class_='search-rst clearfix'):
        item = list()
        item.append(div.h3.a['title'])
        item.append(div.find('span', class_='num').text)
        if div.find('span', 'platform-name'):
            item.append(div.find('span', 'platform-name').text.strip())
        else:
            item.append('無')
        items.append(item)
    print('共 %d 項商品' % (len(items)))
    print('查詢結果：')
    for item in items:
        print(item)
    print()
    return items

# 建立更動db指令的function
def execute_db(fname, sql_cmd):
    conn = sqlite3.connect(fname)
    c = conn.cursor()
    c.execute(sql_cmd)
    conn.commit()
    conn.close()

# 建立查詢db指令的function
def select_db(fname, sql_cmd):
    conn = sqlite3.connect(fname)
    c = conn.cursor()
    c.execute(sql_cmd)
    rows = c.fetchall()
    conn.close()
    return rows

# 搜尋結果輸出成csv檔的function
def csv_w():
    #linux, mac encoding='utf-8', windows 'cp950'
    with open('ezprice_results.csv', 'w', encoding='cp950', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(('商品', '價格', '店家'))
        for item in items:
            writer.writerow([column for column in item])
    return None

# 讀取csv檔的function
def csv_r():
    print('-----讀取 csv 檔-----')
    with open('ezprice_results.csv', 'r', encoding='cp950') as f:
        reader = csv.DictReader(f)
        for row in reader:
            print(row['商品'], row['價格'], row['店家'])
    return None

In [3]:
if __name__ == '__main__':
    
    items = main()
    csv_w()
    csv_r()
    print()
    

    db_name = 'ezprice_results.sqlite'
    print('建立資料庫及資料表')
    cmd = 'CREATE TABLE results (id INTEGER PRIMARY KEY AUTOINCREMENT, item TEXT, price INTEGER, shop TEXT)'
    execute_db(db_name, cmd)
    
    print('插入多筆資料')
    with open('ezprice_results.csv', 'r', encoding='cp950') as f:
        reader = csv.DictReader(f)
        for row in reader:
            cmd = 'INSERT INTO results (item, price, shop) VALUES ("%s", %d, "%s")' % (row['商品'], int(re.findall('[0-9]*', row['價格'].replace(',',''))[0]), row['店家'])
            execute_db(db_name, cmd)

    print('查詢選擇資料')
    cmd = 'SELECT * FROM results WHERE shop="friDay購物"'
    for row in select_db(db_name, cmd):
        print(row)
        
    '''    
    # 其他指令測試測試
    print('插入測試資料')
    cmd = 'INSERT INTO results (item, price, shop) VALUES ("PS4測試機", 1000, "測試賣家")'
    execute_db(db_name, cmd)

    print('更新資料')
    cmd = 'UPDATE results SET shop="iii 賣家" where shop="測試賣家"'
    execute_db(db_name, cmd)
    '''
    

輸入商品：任天堂 switch
query url: %E4%BB%BB%E5%A4%A9%E5%A0%82%20switch


共 14 項商品
查詢結果：
['【任天堂 nintendo】switch 1-2-Switch', '1,490', 'udn買東西']
['NS 任天堂 Nintendo Switch 1-2-Switch', '1,490', 'friDay購物']
['任天堂 Nintendo Switch 1-2-Switch(HAC-P-AACCA(CHT))', '1,490', '燦坤']
['【任天堂 nintendo】switch 寶可拳', '1,790', 'udn買東西']
['任天堂 Nintendo Switch Pro控制器', '2,050', 'friDay購物']
['任天堂 Nintendo Switch 專用 SanDisk microSDXC 64GB 記憶卡', '1,880', 'myfone購物']
['SanDisk 任天堂 Nintendo SWITCH 專用 microSDXC 64GB 記憶卡', '1,880', 'friDay購物']
['Nintendo 任天堂 NS 1-2-Switch 日文版', '1,590', 'isunfar愛順發3C購物網']
['SanDisk 任天堂 Nintendo SWITCH 專用 microSDXC 128GB 記憶卡', '3,680', 'friDay購物']
['任天堂Nintendo Switch-灰 Joy-Con', '9,780', 'Yahoo奇摩購物中心']
['【Nintendo 任天堂】NS Switch 地產大亨 Monopoly for Nintendo Switch(日文版)', '1,390', 'momo富邦購物館']
['NS 任天堂 Nintendo Switch 地產大亨 日文版 Monopoly for Nintendo Switch', '1,490', '博客來']
['【Nintendo 任天堂】NS Switch 1-2-Switch(日文版)', '1,390', 'momo富邦購物館']
['任天堂Nintendo Switch-電光藍/紅 Joy-Con', '9,780', 'Yahoo奇摩購物中心']

-----讀取 csv 檔-----
【任天堂 nintendo】switch 1-2-Switch 1,490 ud